# Data simulation with survivalGPU 

The survivalGPU package allow to simulate data according to the permutation algorithm. 




In [4]:
import torch
import os
import sys
import pykeops

sys.path.append("dev/survivalGPU/python")
os.getcwd()
import survivalgpu
# from survivalgpu import use_cuda, device, float32, int32, int64
# from survivalgpu.utils import numpy
# from survivalgpu import wce_torch








In [22]:
from survivalgpu.simulation import simulate_dataset
from survivalgpu.wce import wce_torch
from survivalgpu.utils import device, float32, int32, int64


dataset = simulate_dataset(max_time = 365, 
                           n_patients = 1000, 
                           doses = [1,1.5,2,2.5,4], 
                           scenario = "exponential_scenario", 
                           HR_target = 1.5)
print(dataset)

        patient  start  stop  event  dose
0             1      0     1      1   4.0
1             2      0     1      0   2.0
2             3      0     1      0   2.5
3             3      1     2      0   2.5
4             4      0     1      0   1.0
...         ...    ...   ...    ...   ...
155700     1000    358   359      0   0.0
155701     1000    359   360      0   0.0
155702     1000    360   361      0   0.0
155703     1000    361   362      0   0.0
155704     1000    362   363      1   0.0

[155705 rows x 5 columns]


In [38]:
from survivalgpu.simulation import simulate_dataset
from survivalgpu.wce import wce_torch
from survivalgpu.utils import device, float32, int32, int64


dataset = simulate_dataset(max_time = 365, 
                           n_patients = 1000, 
                           doses = [1,1.5,2,2.5,4], 
                           scenario = "exponential_scenario", 
                           HR_target = 1.5)
print(dataset)


patient = torch.tensor(dataset["patient"], device = device, dtype = int32)
start = torch.tensor(dataset["start"], device = device, dtype = int32)
stop = torch.tensor(dataset["stop"], device = device, dtype = int32)
events = torch.tensor(dataset["event"], device = device, dtype = int32)
doses = torch.tensor(dataset["dose"], device = device, dtype = float32)


nknots = 3
batchsize = 10
n_bootsraps = 10
constraint = "Right"
cutoff = 180

result = wce_torch(ids = patient, doses = doses, events = events, times = start,
                          cutoff = cutoff, nknots = nknots,covariates = None, 
                          batchsize = batchsize, bootstrap = n_bootsraps, constrained = constraint,
                          verbosity = 0
                          )






        patient  start  stop  event  dose
0             1      0     1      1   4.0
1             2      0     1      1   1.0
2             3      0     1      0   2.0
3             3      1     2      1   2.0
4             4      0     1      0   4.0
...         ...    ...   ...    ...   ...
155634     1000    360   361      0   0.0
155635     1000    361   362      0   0.0
155636     1000    362   363      0   0.0
155637     1000    363   364      0   0.0
155638     1000    364   365      1   0.0

[155639 rows x 5 columns]
[KeOps] Generating code for Sum_Reduction reduction (with parameters 0) of formula (Extract(BSpline_Impl(a,b-c,0),0,1)*Extract(BSpline_Impl(d,b-c,3),0,7))*e with a=Var(0,2,2), b=Var(1,1,0), c=Var(2,1,2), d=Var(3,11,2), e=Var(4,1,2) ... OK
[pyKeOps] Compiling pykeops cpp 8f9375c4d5 module ... OK


In [44]:
print(result.keys())
print(result["WCEmat"])

dict_keys(['hessian', 'sctest init', 'loglik init', 'loglik', 'coef', 'u', 'imat', 'means', 'knotsmat', 'std', 'SED', 'WCEmat', 'est', 'vcovmat'])
tensor([[ 1.0160e-02,  9.8221e-03,  9.4935e-03,  ...,  4.8622e-06,
          1.8014e-06,  2.5809e-07],
        [ 1.4055e-02,  1.3278e-02,  1.2533e-02,  ...,  1.2389e-05,
          4.6181e-06,  6.6367e-07],
        [ 1.3321e-02,  1.2560e-02,  1.1835e-02,  ...,  9.0931e-06,
          3.3798e-06,  4.8500e-07],
        ...,
        [ 7.1780e-03,  7.2717e-03,  7.3399e-03,  ...,  2.3710e-06,
          8.6515e-07,  1.2298e-07],
        [ 6.2455e-03,  6.5616e-03,  6.8452e-03,  ...,  2.1184e-05,
          7.8993e-06,  1.1354e-06],
        [ 1.6370e-02,  1.5401e-02,  1.4476e-02,  ..., -3.8800e-06,
         -1.4587e-06, -2.1052e-07]])
